### Лекция 3. Классы. Введение

Класс и объект - ключевые термины в ООП - языках

![](classes.jpg)

* Барсик - объект класса "Кошка домашняя"
* "Кошка домашняя" - класс, наследник класса "Кошки"
* "Кошки" - класс, наследник класса "Кошачьи"
* "Кошачьи" - класс, наследник класса "Хищные"


Объект класса - конкретный элемент этого типа.

Класс-наследник __уточняет__ характеристики базового класса

Для описания классов в языке С++ есть два ключевых слова: `class` и `struct`

```c++
class Animal
{
    ...
};

struct Point
{
    ...
};
```

Класс может иметь данные: члены класса

```c++
class Animal
{
    std::string name;
    unsigned age;
};

struct Point
{
    float x;
    float y;
};
```

Как создавать объекты таких классов:

```c++
Animal a;
Point p;
```

Доступ до полей:

```c++
p.x = 3.f;
p.y = 4.f;
```

Класс может иметь методы:

```c++
class Animal
{
    bool is_too_young() const
    {
        return age == 0;
    }
   
    std::string name;
    unsigned age;
};

//
// или так:
//

class Animal
{
    bool is_too_young() const;
    
    std::string name;
    unsigned age;
};

bool Animal::is_too_young() const
{
    return age == 0;
}
```

Класс может наследоваться от другого класса:

```c++
class Cat : Animal
{
};

struct WeightedPoint : Point
{
    float w;
};
```

<br />

У членов и методов классов есть модификаторы видимости:
* `private` - доступ только себе самому и друзьям
* `protected` - плюс наследники
* `public` - кто угодно

Отличие `class` от `struct` (с точки зрения компилятора) - модификатор видимости по умолчанию для методов/членов и при наследовании.
* `class` -> `private`
* `struct` -> `public`

```c++
class Animal
{
public:
    bool is_too_young() const { return age == 0; }
  
private:
    std::string name;
    unsigned age;
};

class Cat : public Animal
{
public:
    void say_meow() const { std::cout << "meow"; }
};

struct Point
{
    float x;
    float y;
};

struct WeightedPoint : Point
{
    float w;
};
```

```c++
Cat barsic;
barsic.say_meow();      // OK
barsic.name = "Barsic"; // Compile-time error: name is private member
    
WeightedPoint p;
p.x = 3.f;  // OK
p.w = 0.f;  // OK
```

<br />

Иногда приходится создавать объекты классов, копировать и уничтожать их. Для этого существуют конструкторы и деструкторы:

Для каждого класса могут быть определены (или удалены) основные операции:
* конструктор по умолчанию (default constructor)
* копирующий конструктор (copy constructor)
* перемещающий конструктор (move constructor)
* копирующий оператор присваивания (copy assignment)
* перемещающий оператор присваивания (move assignment)
* деструктор (destructor)

```c++
class Animal
{
private:
    std::string name;
    unsigned age;
    
public:
    // конструктор по умолчанию
    //
    // Animal an;
    Animal() : age(0) {}
    // что с name?

    // конструктор из одного аргумента
    //
    // Animal an("Murka");
    Animal(const std::string& a_name)
        : name(a_name)
        , age(0)
    {}

    // конструктор из двух аргументов
    //
    // Animal an("Murka", 7);
    Animal(const std::string& a_name, const int a_age)
        // список инициализации для членов класса
        // плюс, возможно, спецификация вызова базового класса
        : name(a_name)
        , age(a_age)
    {
        // тело конструктора
        std::cout << "creating animal" << std::endl;
        std::cout << "animal created" << std::endl;
    }  // когда выполнение доходит до конца конструктора, объект класса Animal считается сконструированным
    
    // конструктор копирования
    //
    // Animal a1;  // default
    // Animal a2 = a1;
    // Animal a3(a2);
    // имеет доступ к приватным полям rhs, т.к. объект того же класса
    Animal(const Animal& rhs)
        : name(rhs.name)
        , age(rhs.age)
    {}
    
    // конструктор перемещения
    //
    // Animal a1("Murka", 13);
    // Animal a2 = std::move(a1);
    // a2 - ? a1 - ?
    Animal(Animal&& rhs)
        : name(std::move(rhs.name))
        , age(rhs.age)
    {}
    
    // копирующее присваивание
    //
    // Animal a1("Murka", 13), a2;
    // a2 = a1;
    Animal& operator = (const Animal& rhs)
    {
        ...;
        return *this;
    }
    
    // перемещающее присваивание
    //
    // Animal a1("Murka", 13), a2;
    // a2 = std::move(a1);
    Animal& operator = (Animal&& rhs)
    {
        ...;
        return *this;
    }
    
    // деструктор
    //
    // обратить внимание, что в деструкторе происходит больше операций, чем вывод, объяснить, почему.
    ~Animal()
    {
        std::cout << "destroying " << name << std::endl;
    }
};
```

<br />

##### Когда вызываются конструкторы и деструкторы

Конструктор класса вызывается в момент его создания. Пока конструктор класса не отработает, объектом пользоваться нельзя.

Деструктор класса вызывается, когда объект покидает свою область видимости.

```c++
void fun()
{
    Grasshopper barsic1("Barsic", 1);
    
    {
        Grasshopper barsic2("Barsic", 2);
    }
    
    Grasshopper barsic3("Barsic", 3);
}
```

```c++
void fun()
{
    Grasshopper* barsic = new Grasshopper("Barsic", 1);
    
    for (int i = 0; i < 10; ++i)
    {
        Grasshopper barsic2("Barsic", i);
    }
}
// Обсудить:
//    1. последовательность конструкторов-деструкторов
//    2. какая проблема
//    3. как поправить
```

```c++
void fun()
{
    Grasshopper* barsics = new Grasshopper[3];
    
    if (Grasshopper h = make_grasshopper(); h.is_alive())
    {
        std::cout << "it is alive!" << std::endl;
    }
    else
    {
        std::cout << "" << std::endl;
    }
    
    delete barsics;
}
// Обсудить:
//    1. последовательность конструкторов-деструкторов
//    2. какая проблема
//    3. как поправить
```

<br />

##### RAII (Resource Acqusition Is Initialization)

RAII - ключевая идиома в С++: захват ресурса должен совпадать с инициализацией объекта, а освобождение ресурса с его разрушением.

Т.о.:
* конструктор объекта захватывает ресурс
* деструктор объекта его освобождает

**Пример 1**: массив на куче

```c++
class ArrayD
{
private:
    double* data;
    int size;
    
public:
    ArrayD(int n)
        : data(new double[n]),
        , size(n)
    {
    }
    
    ~ArrayD()
    {
        delete[] data;
    }    
};

void func()
{
    ArrayD arr(100);    
    ...
}
```

Теперь вы можете представить как работает такой код с памятью и почему он корректен:

```c++
void fun()
{
    std::vector<std::string> team = {"Dobrynia", "Ilusha", "Alesha"};
    
    for (const auto& member : team)
        std::cout << member << std::end;    
}
```

**Пример 2**: С++-обёртка над С-файлом

```c++
class File
{
private:
    FILE* file;
    
public:
    File(const char* filename) : file(std::fopen(filename, "w+"))
    {
    }
    
    ~File()
    {
        if (file)
            std::fclose(file);
    }
    ...
};


void fun()
{
    File f("input.txt");
    ...
}
```

<br />

##### Какую часть работы по составлению конструкторов, деструкторов и операторов присваивания можно отдать компилятору?

Компилятор может сгенерировать эти операции в дефолтном виде **по полям**, правила такие:

Что имеется ввиду под фразой **по полям** :

```c++
class Animal
{
    std::string name = "UNK";
    int age = 0;
};

Animal an1;           // значение?
Animal an2(an1);      // значение?
an1 = an2;            // значение?
an2 = std::move(an1); // значение?
```

* автогенерируемый дефолтный конструктор устанавливает полям дефолтные значения
* автогенерируемый конструктор копирования копирует значения по полям
* автогенерируемый конструктор перемещения перемещает значения по полям
* автогенерируемый оператор присваивания ... (вопрос к студентам)
* автогенерируемый оператор перемещения ... (вопрос к студентам)
* автогенерируемый деструктор ... (вопрос к студентам)

__Вопрос__: В каких случаях нужно избежать автогенерированного конструктора копирования? Пример?

<details>
<summary>Ответ 1</summary>
<p>

Когда копирование по полям некорректно копирует состояние класса. Пример:

```c++
class String
{
    const char* data;
    unsigned int len;
};
```

</p>
</details>

<details>
<summary>Ответ 2</summary>
<p>

Когда копирование запрещено. Пример:

```c++
class File
{
    FILE* file;
};
```

</p>
</details>

__Вопрос для продвинутых__: В каких ещё случаях (кроме ручного deepcopy) нужно избежать автогенерированного конструктора перемещения? Пример?

<details>
<summary>Ответ</summary>
<p>

Когда есть зависимость по полям. Пример:


```c++
class ListWithZerosCount
{
    std::list<int> list;
    int zeros_count;
};


ListWithZerosCount l1 = ...;

ListWithZerosCount l2 = std::move(l1);

// можем продолжить работу с l1
```

Объяснение: перемещение всегда оставляет перемещаемый объект в unspecified valid state (если специально не оговорено иначе). Если по полям есть зависимость, перевод всех полей в unspecified valid state mможет нарушить корректность самого класса.

</p>
</details>


<br />

Правила автогенерации:

![](comp_rules.jpg)

Но никто их не помнит наизусть, поэтому работает правило 6:
    
__Определил одно - определи все шесть__

<br />

**Замечание:** далее рисовать происходящее в памяти, так понимается лучше

<br />

##### Мой первый класс

Реализуем класс - строку на С++.

Важнейший принцип языка, основа его основ - RAII (Resource Acquisition Is Initialization)

```c++
class String
{
public:
    String();
    String(const char *s);
    String(const char *s, int size);
    ~String();
    String(const String& rhs);
    String(String&& rhs);

    String& operator = (const String& rhs);
    String& operator = (String&& rhs);

    friend String operator + (const String& lhs, const String& rhs);

private:
    char* s_;  // pointer to null-terminated characters
    size_t l_; // strlen(s_) == l_
};
```

Для начала реализуем оператор сложения двух строк

```c++
String operator + (const String& lhs, const String& rhs)
{
    const size_t res_size = lhs.l_ + rhs.l_;
    const char* res_s = new char[res_size + 1];
    strcpy(res_s, lhs.s_);
    strncpy(res_s + lhs.l_, rhs.s_, rhs.l_);
    
    String s(res_s, res_size);

    delete[] res_s;
    return s;
}
```

**конструкторы** - код, вызываемый при создании объекта

```c++
String::String()
    : s_(new char[1])
    , l_(0)
{
    s_[0] = 0;
}

String::String(const char* s)
{
    l_ = strlen(s);
    s_ = new char[l + 1];
    strcpy(s_, s);
}

String::String(const char *s, int size)
    : s_(new char[size + 1])
    , l_(size + 1)
{
    strncpy(s_, s, size);
    s_[l_] = 0;
}
```

**деструктор** - код, который будет вызываться при уничтожении объекта

```c++
String::~String()
{
    delete[] s_;
}
```

Остановимся на секунду и всомним про RAII

**конструктор копирования**

```c++
String::String(const String& rhs)
{
    s_ = new char[rhs.l_ + 1];
    l_ = rhs.l_;
    strcpy(s_, rhs.s_);
}
```

**конструктор перемещения**

```c++
String::String(String&& rhs)
    : s_(rhs.s_)
    , l_(rhs.l_)
{
}
```

Что-то пошло не так. Что именно?

Не работает такой код, а должен:

```c++
{
    String s1 = "run, Forest, run!";
    String s2 = std::move(s1);
}
```

Реализуем правильный конструктор перемещения

```c++
String::String(String&& rhs)
    : s_(rhs.s_)
    , l_(rhs.l_)
{
    rhs.s_ = 0;
    rhs.l_ = 0;
}
```

А точно ли он правильный?

<details>
<summary>Ответ:</summary>

rhs сломан

</details>

Третья попытка реализовать правильный конструктор перемещения

```c++
String::String(String&& rhs)
{
    s_ = rhs.s_;
    l_ = rhs.l_;

    rhs.s_ = new char[1];
    rhs.s_[0] = 0;
    rhs.l_ = 0;
}
```

**копирующее присваивание**

```c++
String& String::operator =(const String& rhs)
{
    s_ = new char[rhs.l_ + 1];
    l_ = rhs.l_;
    strcpy(s_, rhs.s_);

    return *this;
}
```

<details>
<summary>Где ошибка?</summary>

Утекает предыдущий массив `s_`

</details>

```c++
String& String::operator =(const String& rhs)
{
    delete[] s_;

    s_ = new char[rhs.l_ + 1];
    l_ = rhs.l_;
    strcpy(s_, rhs.s_);

    return *this;
}
```

<details>
<summary>Где ошибка?</summary>

Самоприсваивание

</details>

Правильная реализация будет выглядеть так:

```c++
String& String::operator =(const String& rhs)
{
    if (this != &rhs)
    {
        delete[] s_;

        s_ = new char[rhs.l_ + 1];
        l_ = rhs.l_;
        strcpy(s_, rhs.s_);
    }
    return *this;
}
```

**перемещающее присваивание**:

**Вариант 1:** с очисткой `rhs`

```c++
String& String::operator =(String&& rhs)
{
    if (this != &rhs)
    {
        delete[] s_;

        s_ = rhs.s_;
        l_ = rhs.l_;

        rhs.s_ = new char[1];
        rhs.s_[0] = 0;
        rhs.l_ = 0;
    }

    return *this;
}
```

**Вариант 2:** обмен с `rhs`

```c++
String& String::operator =(String&& rhs)
{
    std::swap(s_, rhs.s_);
    std::swap(l_, rhs.l_);
    return *this;
}
```

<br />

##### Порядок конструирования

```c++
#include <iostream>

class Name {
public:
    Name() { std::cout << "Name "; }
    ~Name() { std::cout << "~Name "; }
};

class Leg {
public:
    Leg() { std::cout << "Leg "; }
    ~Leg() { std::cout << "~Leg "; }
};

class Hat {
public:
    Hat() { std::cout << "Hat "; }
    ~Hat() { std::cout << "~Hat "; }
};

class Animal {
public:
    Animal() { std::cout << "Animal "; }
    ~Animal() { std::cout << "~Animal "; }

private:
    Name name_;
};

class Turtle : public Animal {
public:
    Turtle() { std::cout << "Turtle "; }
    ~Turtle() { std::cout << "~Turtle "; }

private:
    Leg l1_, l2_, l3_, l4_;
    Hat hat_;
};

// show this later
// Animal animal;

int main() {
    std::cout << "hello!" << std::endl;
    Turtle tortilla;
    std::cout << std::endl << "goodbye!" << std::endl;
    return 0;
}
```

Порядок вызова конструкторов:
1. сначала базовый класс
2. потом члены в порядке их объявления
3. потом конструктор самого класса

Порядок вызова деструкторов:
* обратно порядку вызова конструкторов

Ожидаемый вывод:

```sh
hello!
Name Animal Leg Leg Leg Leg Hat Turtle
goodbye!
~Turtle ~Hat ~Leg ~Leg ~Leg ~Leg ~Animal ~Name
```

<br />